# Supervised learning: Regression
<b>Problem statement</b>:<br>
We are given a collection of samples $\{(\mathbf{x}_i,y_i)\}_{i=1}^N$, where $N$  is the size of the collection, $\mathbf{x}_i$ is the sample vector containing $D$ features, and $y_i$ is a <b>real-valued</b> target attribute. The goal is to find a mathematical function which given a vector $\mathbf{x}_k$ of features will output the target value $y_k$.

In <b>Linear Regression</b> we assume that this mapping function has the following form:
$$f(\mathbf{x})=\mathbf{w}\mathbf{x} + b$$

Here $\mathbf{w}$ is a $D$-dimensional vector of parameters (weights) and $b$ is a constant.<br>
The goal is to discover (learn) $f$ from data. Then we can use $f$ to predict $y$ given the feature vector for a new, unknown observation $\mathbf{x}$: $y \leftarrow f(\mathbf{x})$.<br>
For example, if $\mathbf{x}$ has only a single feature (dimension) then we are looking for an equation of the line which fits the relationship between  $\mathbf{x}$ and $y$ as close as possible:
<img src="images/linear_regr_example.png">

<b>Solution:</b><br>
Thus, to find coefficients $\mathbf{w}$ and $b$ of the best-fitting line, we need to minimize the following expression:

$$SSR=\frac{1}{N} \Sigma_{i=1}^N(f_{\mathbf{w},b}(x_i) - y_i)^2$$

The expression we are trying to minimize is called an <em>objective function</em>. The expression $(f(x_i) - y_i)^2$ is called the <em>loss function</em>. It is an absolute difference between the value predicted by the model and the empirical observation. 

In Linear Regression, the loss function is also called the $SSR$ -- the average of the <b>Sum of Squared Residuals</b> computed by the above formula. $SSR$ averages all discrepancies between the model and observed data points.<br>

The loss function above is easily differentiable: if we calculate its derivative (gradient) and set it to zero, then we can just solve a system of linear equations to get optimal values of $\mathbf{w}$ and $b$.
(As we know from Calculus: to find the minimum or the maximum of a function, we set the gradient to zero because the value of the gradient at extrema of a function is always zero.)

## Predicting home prices
Given a vector containing features of a house, we want to be able to predict its price. To build a predictive regression model, we use the dataset ''House Sales in King County, USA'', downloaded from kaggle.<br>

Download the data file [housing.csv](https://docs.google.com/spreadsheets/d/1vk06vuH277_905XORBYQhc4Cf8Poe3BS0AsiBmlzcKA/edit?usp=sharing) to your local directory.<br>
__Update the variable `file_name` in the cell below to point to your local directory where you store the datasets for this course__ and then run the cell.

In [ ]:
file_name = "../../data_ml_2020/housing.csv"

## 1. Simple Linear Regression

### 1.1. Explore the data
After I did some modifications and preprocessing of the original data, we have the following all-numeric features:
<ul>
    <li>id - house identifier, numeric.</li>
    <li>price - house price, numeric. <b>This is the target variable that we are trying to predict</b>.</li>
    <li>bedrooms - no. of bedrooms, numeric.</li>
    <li>bathrooms - no. of bathrooms, numeric.</li>
    <li>sqft_living - square footage of the home, numeric.</li>
    <li>sqft_lot - square footage of the lot, numeric.</li>
    <li>floors - no.of floors, numeric.</li>
    <li>waterfront - has a view to a waterfront, numeric (0 or 1).</li>
    <li>condition - the amount of wear-and-tear, numeric (from 0 to 5).</li>
    <li>sqft_above - square footage of house apart from basement, numeric.</li>
    <li>sqft_basement - square footage of the basement, numeric.</li>
    <li>age - number of years since year built to year sold, numeric.</li>
</ul>

In [ ]:
import pandas as pd
import numpy as np

# this creates a pandas.DataFrame
data = pd.read_csv(file_name, index_col='id')
data.describe()

To make sure that we have some attributes that have a potential of predicting the price and that we are not wasting our time - let's see if there are any features that are significantly correlated with the price:

In [ ]:
# Is there any correlation between features?
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

corr = data.corr()
sns.heatmap(corr)

It seems that <em>sqft_living</em> and <em>sqft_above</em> have 
a positive influence on price, and the house age has the most negative influence.

### 1.2. Build the model

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression(normalize=True)

In [ ]:
Y = data['price'] 
X = data['sqft_living'] 

# convert to numpy vectors (1D vectors in this case)
X=X.values.reshape(len(X),1) 
Y=Y.values.reshape(len(Y),1) 

In [ ]:
# Split the data into training/testing sets 2:1
split_n = len(X)//3 
X_train = X[:-split_n] 
X_test = X[-split_n:] 
   
# Split the targets into training/testing sets 
Y_train = Y[:-split_n] 
Y_test = Y[-split_n:] 

In [ ]:
# Train the model using the training set 
regr.fit(X_train, Y_train)   

In [ ]:
# Plot data points
plt.scatter(X_train, Y_train,  color='black') 
plt.title('Train Data Fit') 
plt.xlabel('Area, sqft') 
plt.ylabel('Price') 

# Plot regression line 
plt.plot(X_train, regr.predict(X_train), color='red',linewidth=3) 
plt.show() 

### 1.3. Model evaluation
The coefficient of determination, denoted as $R^2$, tells you which amount of variation in $𝑦$ is explained by the variance in $\mathbf{𝐱}$, according to our regression model. Larger $R^2$ indicates a better model.<br>
The value $R^2=1.0$ corresponds to $SSR = 0$, that is to the perfect fit since the values of predicted and actual responses fit completely with each other.

In [ ]:
r_sq = regr.score(X_train,Y_train)
print('intercept:', regr.intercept_)
print('slope:', regr.coef_)
print("Coefficient of determination for train data:", r_sq)

How would this model perform on new data?

In [ ]:
r_sq_test = regr.score(X_test,Y_test)
print("Coefficient of determination for test data:", r_sq_test)

### 1.4. Prediction
Now let's take two houses from the dataset and imagine that we want to estimate their prices using our model. The area of the first house is $830$ sqft and the price is \\$85,000. The second house with the area of $7300$ sqft and was sold for \\$5,300,000.


In [ ]:
x = [[830], [7390]]
y = [85000, 5300000]
x, y = np.array(x), np.array(y)
np.set_printoptions(precision=2)

y_pred = regr.predict(x)
print('predicted price:', y_pred)
print('actual price', y)

## 2. Multiple Linear Regression
<em>Multiple</em> or Multivariate Linear Regression is used in cases when we have two or more features to predict the target.

If there are just two features, the regression function becomes: 
$$f(x_1, x_2) = b + w_1x_1 + w_2x_2$$ 
It represents a regression plane in a three-dimensional space.<br> 
The task becomes to determine the values of the $b$, $w_1$ and $w_2$, such that this plane is as close as possible to the actual responses and yields the minimal $SSR$.<br>
In general case with $D>2$ features -- the function has the form:
$$f(x_1, \ldots, x_D) = b + w_1x_1 + \ldots + w_Dx_D$$
and there are $D+1$ parameters to learn. 

### 2.1. Build the model
The process is performed as before, but we use all 10 features as input variables.

In [ ]:
Y = data['price'] 
X = data[['bedrooms','bathrooms', 'sqft_living','sqft_lot',
          'floors', 'waterfront', 'condition', 'sqft_above', 
          'sqft_basement', 'age']] 

X=X.values.reshape(len(X),len(X.columns)) 
Y=Y.values.reshape(len(Y),1) 

# Split the data into training/testing sets 
split_n = 2*len(X)//3 
X_train = X[:-split_n] 
X_test = X[-split_n:] 
   
# Split the targets into training/testing sets 
Y_train = Y[:-split_n] 
Y_test = Y[-split_n:] 

regr = linear_model.LinearRegression(normalize=True)
regr.fit(X_train, Y_train)

### 2.2. Evaluate multivariate model
Does this model predict better than the single-variable model?

In [ ]:
r_sq_train = regr.score(X_train,Y_train)
print("Coefficient of determination for train data:", r_sq_train)
print('intercept:', regr.intercept_)
print('slope:', regr.coef_)

r_sq_test = regr.score(X_test,Y_test)
print("Coefficient of determination for test data:", r_sq_test)

If $R^2$ score is significantly better for the train data than for the test data - this is a sign of overfitting: we followed too closely to the training data points and the model failed to generalize. 

### 2.3. Prediction
What about our two houses?

In [ ]:
x = [[2,1,830,9000,1,0,3,830,0,75], [6,6,7390,24829,2,1,4,5000,2390,24]]
y = [85000, 5300000]
x, y = np.array(x), np.array(y)

y_pred = regr.predict(x)
print('predicted price:', y_pred)
print('actual price', y)

## 3. Polynomial Regression

In this case, we assume that the dependence between input and target variables is not linear, but polynomial. 

In other words, in addition to linear terms like $w_1x_1$, our regression function $f$ can include non-linear terms such as $w_2x_1^2$, $w_3x_1^3$, or even $w_4x_1x_2$, $w_5x_1^2x_2$, and so on.

The simplest example of polynomial regression in the case of a single input variable is a polynomial of degree 2: 
$$f(x) = b + w_1x + w_2x^2$$.

Now, we want to compute $b$, $w_1$ and $w_2$.  
Keeping this in mind, compare the polynomial regression function with the function $f(x) = b + w_1x_1 + w_2x_2$ used for linear regression. They look very similar and are both linear functions with the unknown coefficients $b$, $w_1$ and $w_2$. This is why we can solve the polynomial regression problem as a linear problem with the term $x^2$ regarded as an additional input variable.

In the case of two variables and the polynomial of degree 2, the regression function has this form: 

$$f([x_1, x_2]) = b + w_1x_1 + w_2x_2 + w_3x_1^2 + w_4x_2^2  + w_5x_1x_2$$. 

The algorithm for solving this problem is exactly the same: we apply linear regression learning to 5 input variables: $x_1$, $x_2$, $x_1^2$, $x_2^2$, and $x_1x_2$.  As the result we get the values of six weights which minimize $SSR$: $b$, $w_1$, $w_2$, $w_3$, $w_4$ and $w_5$.

### 3.1. Adding derived input variables
All we need to do, is to add some more artificial columns to the dataset. We use the <code>PolynomialFeatures</code> of the <code>sklearn.preprocessing</code> package. 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

### 3.2. Polynomial regression with a single variable
Let's start with a single input variable -- <code>sqft_living</code>, and the polinomial of degree 2.

In [ ]:
Y = data['price'] 
X = data['sqft_living'] 

# convert to numpy vectors (1D vectors in this case)
X=X.values.reshape(len(X),1) 
Y=Y.values.reshape(len(Y),1) 
   
# Split the data into training/testing sets 
split_n = 2*len(X)//3 
X_train = X[:-split_n] 
X_test = X[-split_n:] 
   
# Split the targets into training/testing sets 
Y_train = Y[:-split_n] 
Y_test = Y[-split_n:] 

Now let's generate an additional column: $x^2$.

In [ ]:
transformer = PolynomialFeatures(degree=2, include_bias=False)
transformer.fit(X_train)
X_train_ = transformer.transform(X_train)

print(X_train_)

The dataset now contains an additional column $x^2$.<br>
Everything else is performed exactly as before, but we use an extended <code>$X_train_</code> instead of the original <code>$X_train</code>. Note that we have to do the same transformation for the test dataset.

In [ ]:
# Train the model as before
regr = linear_model.LinearRegression(normalize=True)
regr.fit(X_train_, Y_train)    
   
r_sq_train = regr.score(X_train_,Y_train)
print("Coefficient of determination for train data:", r_sq_train)

transformer.fit(X_test)
X_test_ = transformer.transform(X_test)

r_sq_test = regr.score(X_test_,Y_test)
print("Coefficient of determination for test data:", r_sq_test)

### 3.3. Polynomial regression with all the features
Let's try to improve the model using polynomial regression for all the features.

In [ ]:
Y = data['price'] 
X = data[['bedrooms','bathrooms', 'sqft_living','sqft_lot',
          'floors', 'waterfront', 'condition', 'sqft_above', 
          'sqft_basement', 'age']] 

X=X.values.reshape(len(X),len(X.columns)) 
Y=Y.values.reshape(len(Y),1) 

# Split the data into training/testing sets 
split_n = 2*len(X)//3 
X_train = X[:-split_n] 
X_test = X[-split_n:] 
   
# Split the targets into training/testing sets 
Y_train = Y[:-split_n] 
Y_test = Y[-split_n:] 

transformer = PolynomialFeatures(degree=2, include_bias=False)
transformer.fit(X_train)
X_train_ = transformer.transform(X_train)

print(X_train_[0][0], X_train_[0][1], X_train_[0][10], X_train_[0][11] )


The modified input array contains additional columns: one with the original feature, the other with its square plus the multiplications to the values in all other columns.

The model learning algorithm does not change:

In [ ]:
regr = linear_model.LinearRegression(normalize=True)
regr.fit(X_train_, Y_train)

r_sq_train = regr.score(X_train_,Y_train)
print("Coefficient of determination for train data:", r_sq_train)

In [ ]:
transformer.fit(X_test)
X_test_ = transformer.transform(X_test)

r_sq_test = regr.score(X_test_,Y_test)
print("Coefficient of determination for test data:", r_sq_test)

What about our two houses?

In [ ]:
x = [[2,1,830,9000,1,0,3,830,0,75], [6,6,7390,24829,2,1,4,5000,2390,24]]
y = [85000, 5300000]
x, y = np.array(x), np.array(y)

transformer.fit(x)
x_ = transformer.transform(x)

y_pred = regr.predict(x_)
print('predicted price:', y_pred)
print('actual price', y)

That is the best we can do so far. Fortunately, there are other regression techniques suitable for the cases where linear regression doesn’t perform well: Support Vector Regression, Decision Trees, Random Forests, and Neural Networks, to name the few.

Copyright &copy; 2020 Marina Barsky. All rights reserved.